# Creating Flag Parameter

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(incremental_flag)

0


In [0]:
%sql
select * from parquet.`abfss://silver@thadiyalcardatalake.dfs.core.windows.net/carsales`
limit 10

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Day,Month,Year,BranchName,DealerName,Product_Name,model_category,RevPerUnit
BR0001,DLR0001,BMW-M1,13363978,2,DT00001,1,1,2017,AC Cars Motors,AC Cars Motors,BMW,BMW,6681989.0
BR0003,DLR0228,Hon-M218,17376468,3,DT00001,10,5,2017,AC Cars Motors,Deccan Motors,Honda,Hon,5792156.0
BR0004,DLR0208,Tat-M188,9664767,3,DT00002,12,1,2017,AC Cars Motors,Wiesmann Motors,Tata,Tat,3221589.0
BR0005,DLR0188,Hyu-M158,5525304,3,DT00002,16,9,2017,AC Cars Motors,Subaru Motors,Hyundai,Hyu,1841768.0
BR0006,DLR0168,Ren-M128,12971088,3,DT00003,20,5,2017,AC Cars Motors,Saab Motors,Renault,Ren,4323696.0
BR0008,DLR0128,Hon-M68,7321228,1,DT00004,28,4,2017,AC Cars Motors,Messerschmitt Motors,Honda,Hon,7321228.0
BR0009,DLR0108,Cad-M38,11379294,2,DT00004,31,12,2017,AC Cars Motors,Lexus Motors,Cadillac,Cad,5689647.0
BR0010,DLR0088,Mer-M8,11611234,2,DT00005,4,9,2017,AC Cars Motors,"IFA (including Trabant, Wartburg, Barkas) Motors",Mercedes-Benz,Mer,5805617.0
BR0011,DLR0002,BMW-M2,19979446,2,DT00005,2,1,2017,Acura Motors,Acura Motors,BMW,BMW,9989723.0
BR0011,DLR0069,Vol-M256,14181510,3,DT00006,9,5,2017,Acura Motors,Geo Motors,Volkswagen,Vol,4727170.0


#Creating Dimension Model

In [0]:
df_src = spark.sql('''
select Distinct(Date_ID) as Date_ID from parquet.`abfss://silver@thadiyalcardatalake.dfs.core.windows.net/carsales`
''')


In [0]:
df_src.limit(10).display()

Date_ID
DT00029
DT00140
DT00192
DT00444
DT00475
DT00947
DT00976
DT01028
DT01099
DT00657


### dim_branch sink - initial and incremental

In [0]:
if spark.catalog.tableExists('CarProject.gold.dim_date'):
    df_sink = spark.sql('''
                    select dim_date_key, Date_ID from carproject.gold.dim_date
                    ''')
else:
    df_sink = spark.sql('''
                    select 1 as dim_date_key, Date_ID from parquet.`abfss://silver@thadiyalcardatalake.dfs.core.windows.net/carsales`
                    where 1=0
                    ''')


In [0]:
df_sink.display()

dim_date_key,Date_ID


In [0]:
df_filter = df_src.join(df_sink, df_src.Date_ID == df_sink.Date_ID,'left').select(df_src.Date_ID,  df_sink.dim_date_key)
df_filter.limit(10).display()

Date_ID,dim_date_key
DT00029,null
DT00140,null
DT00192,null
DT00444,null
DT00475,null
DT00947,null
DT00976,null
DT01028,null
DT01099,null
DT00657,null


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_filter_old = df_filter.filter(col('dim_date_key').isNotNull())
df_filter_old.display()

Date_ID,dim_date_key


In [0]:
df_filter_new = df_filter.filter(col('dim_date_key').isNull()).select(df_filter.Date_ID)
df_filter_new.limit(10).display()


Date_ID
DT00029
DT00140
DT00192
DT00444
DT00475
DT00947
DT00976
DT01028
DT01099
DT00657


### Creating Surrogate key

In [0]:
if (incremental_flag == '0'):
    max_value = 1
else:
    max_value_df = spark.sql("select max(dim_date_key) from carproject.gold.dim_date")
    max_value = max_value_df.collect()[0][0]+1

In [0]:
df_filter_new = df_filter_new.withColumn('dim_date_key',max_value + monotonically_increasing_id())


In [0]:
df_filter_new.limit(10).display()

Date_ID,dim_date_key
DT00029,1
DT00140,2
DT00192,3
DT00444,4
DT00475,5
DT00947,6
DT00976,7
DT01028,8
DT01099,9
DT00657,10


### Create Final DF - df_filter_old + df_filter_new

In [0]:
df_final = df_filter_new.union(df_filter_old)
df_final.limit(10).display()

Date_ID,dim_date_key
DT00029,1
DT00140,2
DT00192,3
DT00444,4
DT00475,5
DT00947,6
DT00976,7
DT01028,8
DT01099,9
DT00657,10


### SCD TYPE -1 (UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
# Incremental Run
if spark.catalog.tableExists('CarProject.gold.dim_date'):
    delta_tbl = DeltaTable.forPath(spark,'abfss://gold@thadiyalcardatalake.dfs.core.windows.net/dim_date')
    delta_tbl.alias('trg').merge(df_final.alias('src'), 'trg.dim_date_key = src.dim_date_key') \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()


# Initial Run
else:
    df_final.write.format('delta')\
        .mode('overwrite')\
            .option("path", 'abfss://gold@thadiyalcardatalake.dfs.core.windows.net/dim_date')\
                .saveAsTable('CarProject.gold.dim_date')    

In [0]:
%sql
select * from carproject.gold.dim_date limit 10

Date_ID,dim_date_key
DT00029,1
DT00140,2
DT00192,3
DT00444,4
DT00475,5
DT00947,6
DT00976,7
DT01028,8
DT01099,9
DT00657,10
